<a href="https://colab.research.google.com/github/SA6246/NYU_Tandon_Databases_Spring/blob/main/project/notebooks/project_final1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# Install dependencies
!wget https://raw.githubusercontent.com/SA6246/NYU_Tandon_Databases_Spring/main/project/sources/createdb.sql createdb.sql > /dev/null 2>&1

In [32]:
# Install dependencies
!pip install mysql-connector-python > /dev/null 2>&1

In [33]:
# Install dependencies
!apt-get -y install mysql-server > /dev/null 2>&1

In [34]:
# Start SQL Server
!service mysql start > /dev/null 2>&1

In [35]:
# Create our new Database and create a root account.
!mysql -u root -proot -e "ALTER USER 'root'@'localhost' IDENTIFIED BY 'root'; source createdb.sql;" > /dev/null 2>&1

In [36]:
# Modify Database ROOT user access
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH 'mysql_native_password' BY 'root';FLUSH PRIVILEGES;" > /dev/null 2>&1

In [37]:
# Define the HTML template as a string
# Used a Generic Bootstrap Template

template = """
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>SQL Query</title>
            <!-- Bootstrap CSS -->
            <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
            <style>
                body {
                    padding: 50px;
                }
                .container {
                    max-width: 600px;
                    margin: auto;
                }
            </style>
        </head>
        <body>
            <div class="container">
                <!-- Header -->
                <h1 class="mb-4">Enter Your SQL Query</h1>
                <!-- Instruction -->
                <p class="mb-4">Please enter your SQL query below:</p>
                <!-- Input field for SQL query -->
                <form action="/execute_query" method="post">
                    <div class="form-group">
                        <label for="sqlQuery">SQL Query:</label>
                        <textarea class="form-control" id="sqlQuery" name="sqlQuery" rows="4"></textarea>
                    </div>
                    <!-- Button to submit the form -->
                    <button type="submit" class="btn btn-primary mt-3">Execute Query</button>
                </form>

                <!-- Output Section -->
                <!-- Section to display the output of the query -->
                <div class="mt-5">
                    <h2>Output</h2>
                    <!-- Apply CSS styles for resizing and styling the output -->
                    <div id="output" class="border p-3" style="max-width: fit-content; max-height: 400px; overflow: auto;">{{ output|safe }}</div>
                </div>
                <!-- Operation info Section -->
                <div class="mt-5">
                    <h2>Operation Info</h2>
                    <p>{{ operation_info|safe }}</p>
                </div>

            <!-- Bootstrap JS and dependencies -->
            <script src="https://code.jquery.com/jquery-3.5.1.slim.min.js"></script>
            <script src="https://cdn.jsdelivr.net/npm/@popperjs/core@2.5.4/dist/umd/popper.min.js"></script>
            <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/js/bootstrap.min.js"></script>
        </body>
        </html>
"""

In [38]:
# Expose our notebook endpoint
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://mnrz3h16gpa-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
from flask import Flask, render_template_string, request
import pandas as pd
import mysql.connector

app = Flask(__name__)

@app.route("/")
def home():
    return render_template_string(template)

@app.route("/execute_query", methods=["POST"])
def execute_query():
    # Get the SQL query from the form data
    query = request.form.get("sqlQuery")

    # Pass out command to our query function
    table_html, operation_info = execute_sql_and_display(query)

    # Render the template with the results
    return render_template_string(template,
                                  output=table_html,
                                  operation_info=operation_info)

def execute_sql_and_display(command):

    # Create a connection to the MySQL server
    conn = mysql.connector.connect(user='root',
                                   password='root',
                                   host='localhost')

    # Create a cursor to interact with the MySQL server
    cursor = conn.cursor()

    # Execute the SQL command
    cursor.execute(command)

    # Check operation type
    operation_info = "No specific operation detected"
    if command.strip().lower().startswith("insert"):
        operation_info = "INSERT operation detected"
    elif command.strip().lower().startswith("update"):
        operation_info = "UPDATE operation detected"
    elif command.strip().lower().startswith("delete"):
        operation_info = "DELETE operation detected"
    else:
        operation_info = "READ operation detected"

    # If it's a non-SELECT operation, return operation_info only.
    if command.strip().lower().startswith(("insert", "update", "delete")):
        return None, operation_info

    # Fetch all the results
    records = cursor.fetchall()

    # Create a DataFrame from the records
    df = pd.DataFrame(records)

    # Get column names
    column_names = [col[0] for col in cursor.description]

    # Set column names to the DataFrame
    df.columns = column_names

    # Convert DataFrame to HTML table with column names
    table_html = df.to_html(index=False, header=True)

    return table_html, operation_info

if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [30/Apr/2024 19:47:58] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Apr/2024 19:47:59] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [30/Apr/2024 19:48:11] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Apr/2024 19:48:11] "GET /favicon.ico HTTP/1.1" 404 -
